<a href="https://colab.research.google.com/github/Emmahgithub/Analis-de-rendimientos-y-portafolios/blob/main/Renta_Variable_analisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Analisis de renta variable</i></h1>
  </font>
  <font color="Blue" face="Comic Sans MS,arial">
  <h5 align="center"><i>VARGAS BAUTISTA EMMANUEL</i></h5>
  </font>

In [ ]:
#Librerias
import yfinance as yf
import pandas as pd
import numpy as np

#Calculo de rendimientos y sus estad ́ısticas, ademas de Optimizaci ́on de Portafolios.

In [ ]:
def calcular_estadisticas_en_dataframe(etfs, start_date, end_date):
    # Diccionario para almacenar los DataFrames completos con estadísticas
    resultados = {}

    # Descargar los datos de tipo de cambio USD/MXN
    usd_mxn = yf.download('MXN=X', start=start_date, end=end_date)['Close']

    for etf in etfs:
        # Descargar los precios de cierre del ETF
        etf_data = yf.download(etf, start=start_date, end=end_date)['Close']

        # Unir los dataframes del ETF y del tipo de cambio
        datas = pd.merge(etf_data, usd_mxn, left_index=True, right_index=True, how='inner')

        # Agregar encabezados a las columnas
        datas.columns = ['Precios', 'USD/MXN']

        # Calcular los precios en MXN
        datas['Precios en MXN'] = datas['Precios'] * datas['USD/MXN']

        # Calcular los rendimientos diarios
        datas['Rend'] = datas['Precios en MXN'] / datas['Precios en MXN'].shift(1)

        # Calcular métricas estadísticas
        media = datas['Rend'].mean()
        varianza = datas['Rend'].var()
        desviacion_estandar = datas['Rend'].std()
        sesgo = datas['Rend'].skew()
        exceso_curtosis = datas['Rend'].kurtosis()

        # Calcular rendimiento anualizado
        dias_totales = datas['Rend'].dropna().count()  # Total de días con datos
        rendimiento_anualizado = (np.prod(datas['Rend'].dropna()) ** (252 / dias_totales))

        #Calcular desviación estandar anualizada
        desviacion_anualizada = desviacion_estandar * np.sqrt(252)

        # Agregar métricas como nuevas columnas al DataFrame
        datas['Media'] = media
        datas['Varianza'] = varianza
        datas['Desviación Estándar'] = desviacion_estandar
        datas['Sesgo'] = sesgo
        datas['Exceso de Curtosis'] = exceso_curtosis
        datas['Rendimiento Anualizado'] = rendimiento_anualizado  # Misma columna para todas las filas
        datas['Desviación Anualizada'] = desviacion_anualizada # Misma columna para todas las filas

        # Guardar el DataFrame en el diccionario de resultados
        resultados[etf] = datas

    return resultados

# Lista de ETFs y rango de fechas
etfs = ['EWG', 'EWJ', 'EWU', 'EZU', '^GSPC']  #EWG (Alemania ETF), EWJ (Japon ETF), EWU (Reino Unido ETF), EZU(Zona Euro ETF), Incluyendo el S&P 500 como ^GSPC
start_date = '2010-01-01'
end_date = '2024-12-01'

# Ejecutar la función y obtener resultados
resultados = calcular_estadisticas_en_dataframe(etfs, start_date, end_date)

# Mostrar resultados
for etf, df in resultados.items():
    print(f"\nResultados para {etf}:\n")
    print(df[['Precios', 'USD/MXN', 'Precios en MXN', 'Rend', 'Rendimiento Anualizado', 'Desviación Anualizada']].head())




[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Resultados para EWG:

              Precios  USD/MXN  Precios en MXN      Rend  \
Date                                                       
2010-01-04  22.900000  12.8718      294.764225       NaN   
2010-01-05  22.790001  12.8108      291.958135  0.990480   
2010-01-06  22.930000  12.7395      292.116740  1.000543   
2010-01-07  22.770000  12.7602      290.549750  0.994636   
2010-01-08  22.799999  12.6842      289.199757  0.995354   

            Rendimiento Anualizado  Desviación Anualizada  
Date                                                       
2010-01-04                 1.05559               0.223878  
2010-01-05                 1.05559               0.223878  
2010-01-06                 1.05559               0.223878  
2010-01-07                 1.05559               0.223878  
2010-01-08                 1.05559               0.223878  

Resultados para EWJ:

              Precios  USD/MXN  Precios en MXN      Rend  \
Date                                                 

# Calculo de los rendimientos

In [ ]:
# Crear un DataFrame con rendimientos anualizados y rendimientos diarios (para la matriz de covarianza)
rendimientos_diarios = {}
rendimientos_anualizados = []

etfs_seleccionados = list(resultados.keys())[:4]  # 4 ETFs
for etf in etfs_seleccionados:
    df = resultados[etf]
    rendimientos_diarios[etf] = df['Rend'].dropna()
    rendimientos_anualizados.append(df['Rendimiento Anualizado'].iloc[0])

# Calcular matriz de covarianza (Σ)
rendimientos_diarios_df = pd.DataFrame(rendimientos_diarios)
cov_matriz = rendimientos_diarios_df.cov() * 252  # Anualizar

mu = np.array(rendimientos_anualizados)  # Vector de rendimientos esperados (μ)
unos = np.ones(len(mu))  # Vector de unos

inv_cov = np.linalg.inv(cov_matriz) # Inversa de la matriz de covarianza

# portafolio de mınima varianza y el portafolio tangente a la frontera eficiente.

In [ ]:
lambdas = [1, 2, 7, 9]

def portafolio_media_varianza(mu, cov_matriz, lambdas, tasa_libre_riesgo = 0.0626):

    pesos_optimos = {} # Diccionario para almacenar los resultados de los pesos
    # PESOS OPTIMOS PARA CADA LAMBDA
    for lambda_valor in lambdas:
        w = (1/(lambda_valor)) * inv_cov @ (mu - (tasa_libre_riesgo*unos))
        pesos_optimos[lambda_valor] = w

    return pesos_optimos


# Calcular los pesos óptimos
pesos_optimos = portafolio_media_varianza(mu, cov_matriz, lambdas)

etfs_nombres = ['EWG', 'EWJ', 'EWU', 'EZU']

# Resultados
for lambda_valor, pesos in pesos_optimos.items():
    print(f"\nPesos óptimos para lambda = {lambda_valor}:")
    for etf_nombres, peso in zip(etfs_nombres, pesos):
        print(f"{etf_nombres}: {peso:.4f}")


Pesos óptimos para lambda = 1:
EWG: 5.1811
EWJ: 20.1174
EWU: 14.2979
EZU: -8.2478

Pesos óptimos para lambda = 2:
EWG: 2.5905
EWJ: 10.0587
EWU: 7.1490
EZU: -4.1239

Pesos óptimos para lambda = 7:
EWG: 0.7402
EWJ: 2.8739
EWU: 2.0426
EZU: -1.1783

Pesos óptimos para lambda = 9:
EWG: 0.5757
EWJ: 2.2353
EWU: 1.5887
EZU: -0.9164


In [ ]:
# Función para calcular los pesos óptimos
def calcular_pesos_optimos(mu, cov_matrix, m, rf=0.0626):
    """
    Calcula los pesos óptimos para un portafolio dado el rendimiento esperado m y la tasa libre de riesgo rf.

    Parámetros:
    - mu: vector de rendimientos promedio anualizados de los activos.
    - cov_matrix: matriz de covarianza anualizada de los rendimientos.
    - m: rendimiento esperado del portafolio.
    - rf: tasa libre de riesgo (default=0.1).

    Retorna:
    - w_opt: vector de pesos óptimos.
    """
    # Inverso de la matriz de covarianza
    inv_cov = np.linalg.inv(cov_matriz)

    # Cálculo de estadísticas clave
    A = unos.T @ inv_cov @ unos
    B = mu.T @ inv_cov @ unos
    C = mu.T @ inv_cov @ mu

    # Multiplicadores de Lagrange
    lambda_ = (A * m - B) / (A * C - B**2)
    gamma = (C - B * m) / (A * C - B**2)

    # Pesos óptimos
    w_opt = inv_cov @ (lambda_ * mu + gamma * unos)

    return w_opt

# Rendimiento objetivo
rendimiento_objetivo = 0.13

# Calcular los pesos óptimos
pesos_optimos = calcular_pesos_optimos(mu, cov_matriz, rendimiento_objetivo)
print(pesos_optimos)

# Mostrar los pesos óptimos con el nombre del ETF asociado
etfs_nombres = ['EWG (Alemania)', 'EWJ (Japón)', 'EWU (Reino Unido)', 'EZU (Zona Euro)']

print("\nPesos Óptimos por ETF:")
for etf_nombres, peso in zip(etfs_nombres, pesos_optimos):
    print(f"{etf_nombres}: {peso:.4%}")



[-30.49635708 -16.79068001  25.40145357  22.88558352]

Pesos Óptimos por ETF:
EWG (Alemania): -3049.6357%
EWJ (Japón): -1679.0680%
EWU (Reino Unido): 2540.1454%
EZU (Zona Euro): 2288.5584%


In [ ]:
# Definir A y B
A = unos.T @ inv_cov @ unos
B = mu.T @ inv_cov @ unos

# Portafolio de minima varianza cuando:
m_g = B / A
w_g = (inv_cov @ unos) / A
sigma2_g = 1 / A

# Pesos portafolio tangente
r = 0.0626 # se incluye un activo libre de riesgo
w_t = (inv_cov @ (mu - (r * unos)))/(B - A*r)

print("El portafolio de varianza minima  ocurre cuando m =", m_g)
print("Con pesos w =", w_g , "y σ² =", sigma2_g)
print("\nEl portafolio tangente cuando incluye un activo libre de riesgo tiene pesos w =", w_t)

El portafolio de varianza minima  ocurre cuando m = 1.0569138954350665
Con pesos w = [ 0.08734515  0.59742682  0.5194929  -0.20426487] y σ² = 0.031717906979234906

El portafolio tangente cuando incluye un activo libre de riesgo tiene pesos w = [ 0.16527277  0.64173191  0.45609339 -0.26309807]
